In [ ]:
import pdfplumber
import os
import multiprocessing

def pdf_to_txt_with_pdfplumber(pdf_path, txt_path):
    """PDF 파일을 텍스트로 변환"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([page.extract_text() or "" for page in pdf.pages])  # None 방지
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
    except Exception as e:
        print(f"PDF 변환 오류: {pdf_path} - {e}")

def rename_txt_file_based_on_pdf(pdf_path, txt_path):
    """PDF 파일명을 기반으로 텍스트 파일 이름 변경"""
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    new_name = f"{pdf_name}.txt"
    new_path = os.path.join(os.path.dirname(txt_path), new_name)
    
    try:
        os.rename(txt_path, new_path)
        print(f"파일 이름 변경: {txt_path} → {new_name}")
    except Exception as e:
        print(f"파일 이름 변경 실패 ({txt_path}): {e}")

def process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=4, max_files=5):
    """PDF 폴더 내 모든 파일을 병렬 처리 (테스트용으로 일부만 실행)"""
    os.makedirs(txt_folder_path, exist_ok=True)

    pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
    pdf_files = pdf_files[:max_files]  # 테스트를 위해 일부 파일만 실행 (기본 5개)

    if not pdf_files:
        print("변환할 PDF 파일이 없습니다! `naver_reports_pdf` 폴더에 PDF 파일을 추가하세요.")
        return

    print(f" 총 {len(pdf_files)}개의 PDF 변환을 시작합니다...")

    # 병렬 처리 실행
    with multiprocessing.Pool(processes=num_workers) as pool:
        pool.starmap(process_single_pdf, [(pdf, txt_folder_path) for pdf in pdf_files])

    print(f"테스트 완료! 변환된 TXT 파일 개수: {len(pdf_files)} (저장 폴더: {txt_folder_path})")

def process_single_pdf(pdf_path, txt_folder_path):
    """개별 PDF 파일을 처리"""
    txt_file_name = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(txt_folder_path, txt_file_name)
    
    # PDF → TXT 변환
    pdf_to_txt_with_pdfplumber(pdf_path, txt_path)
    
    # TXT 파일명 변경
    rename_txt_file_based_on_pdf(pdf_path, txt_path)

# **경로 설정 (naver_reports_pdf 폴더 사용)**
pdf_folder_path = './naver_reports_pdf'  # PDF 파일이 저장된 폴더
txt_folder_path = './txt_files'  # 변환된 TXT 파일을 저장할 폴더

# 실행 (테스트: 5개 파일만 변환)
process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=4, max_files=5)


📝 총 5개의 PDF 변환을 시작합니다...


In [1]:
import pdfplumber
import os
import time

# 경로 설정
pdf_folder_path = './naver_reports_pdf'
txt_folder_path = './txt_files'

os.makedirs(txt_folder_path, exist_ok=True)

# PDF 파일 목록 가져오기 (1개만 테스트)
pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
if not pdf_files:
    print("⚠️ 변환할 PDF 파일이 없습니다!")
else:
    pdf_path = pdf_files[0]  # 첫 번째 PDF만 테스트
    txt_path = os.path.join(txt_folder_path, os.path.splitext(os.path.basename(pdf_path))[0] + ".txt")
    
    print(f"🚀 테스트 실행: {pdf_path}")
    
    start_time = time.time()  # 실행 시간 체크
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join([page.extract_text() or "" for page in pdf.pages])  # None 방지
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)
    
    end_time = time.time()
    print(f"✅ 변환 완료: {txt_path} (소요 시간: {end_time - start_time:.2f}초)")


🚀 테스트 실행: ./naver_reports_pdf\080401_대우증권_1.pdf
✅ 변환 완료: ./txt_files\080401_대우증권_1.txt (소요 시간: 6.36초)


In [ ]:
import pdfplumber
import os
import multiprocessing
import time

def pdf_to_txt_with_pdfplumber(pdf_path, txt_path):
    """PDF 파일을 텍스트로 변환 (속도 개선)"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # PDF 페이지 개수 체크 (너무 많으면 제외)
            if len(pdf.pages) > 50:  # 50페이지 이상이면 변환 제외 (필요시 조정)
                print(f"⚠️ 페이지가 너무 많아 변환 제외: {pdf_path}")
                return

            # 더 빠르게 추출하는 방식 적용
            text = "\n".join([page.extract_text() or "" for page in pdf.pages if page.extract_text()])
        
        # 변환된 텍스트 저장
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    except Exception as e:
        print(f"❌ PDF 변환 오류: {pdf_path} - {e}")

def process_single_pdf(pdf_path, txt_folder_path):
    """개별 PDF 파일을 처리"""
    txt_file_name = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(txt_folder_path, txt_file_name)
    
    start_time = time.time()
    pdf_to_txt_with_pdfplumber(pdf_path, txt_path)
    end_time = time.time()

    print(f"✅ 변환 완료: {txt_path} (소요 시간: {end_time - start_time:.2f}초)")

def process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=4, max_files=5):
    """PDF 폴더 내 모든 파일을 병렬 처리"""
    os.makedirs(txt_folder_path, exist_ok=True)

    pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
    pdf_files = pdf_files[:max_files]  # 테스트용 파일 개수 제한

    if not pdf_files:
        print("⚠️ 변환할 PDF 파일이 없습니다!")
        return

    print(f"📝 총 {len(pdf_files)}개의 PDF 변환을 시작합니다...")

    # 병렬 처리 실행 (적절한 workers 값 설정)
    with multiprocessing.Pool(processes=num_workers) as pool:
        pool.starmap(process_single_pdf, [(pdf, txt_folder_path) for pdf in pdf_files])

    print(f"✅ 변환 완료! (저장 폴더: {txt_folder_path})")

# **경로 설정**
pdf_folder_path = './naver_reports_pdf'  # PDF 파일이 저장된 폴더
txt_folder_path = './txt_files'  # 변환된 TXT 파일을 저장할 폴더

# 실행 (5개 파일 테스트, num_workers 최적화)
process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=2, max_files=5)


📝 총 5개의 PDF 변환을 시작합니다...


In [2]:
!pip install pytesseract pdf2image


In [3]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import os
import time
from concurrent.futures import ThreadPoolExecutor

# OCR을 위한 Tesseract 설치 경로 설정 (Windows 사용 시 필요)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # 실제 설치 경로 입력

def pdf_to_txt(pdf_path, txt_path):
    """PDF를 텍스트로 변환 (텍스트 PDF + OCR 지원)"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([page.extract_text() or "" for page in pdf.pages if page.extract_text()])

        # 🔥 OCR 적용: PDF가 텍스트를 포함하지 않는 경우 OCR 사용
        if not text.strip():
            print(f"⚠️ {pdf_path}는 이미지 PDF일 가능성이 높음 → OCR 실행")
            text = ocr_pdf(pdf_path)

        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    except Exception as e:
        print(f"❌ PDF 변환 오류: {pdf_path} - {e}")

def ocr_pdf(pdf_path):
    """OCR을 사용하여 이미지 PDF에서 텍스트 추출"""
    try:
        images = convert_from_path(pdf_path)
        text = "\n".join([pytesseract.image_to_string(img, lang='kor+eng') for img in images])
        return text
    except Exception as e:
        print(f"❌ OCR 변환 오류: {pdf_path} - {e}")
        return ""

def process_single_pdf(pdf_path, txt_folder_path):
    """PDF 하나를 변환하는 함수"""
    txt_file_name = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(txt_folder_path, txt_file_name)

    start_time = time.time()
    pdf_to_txt(pdf_path, txt_path)
    end_time = time.time()

    print(f"✅ 변환 완료: {txt_path} (소요 시간: {end_time - start_time:.2f}초)")

def process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=4, max_files=5):
    """PDF 폴더 내 모든 파일을 병렬 처리 (멀티쓰레딩 사용)"""
    os.makedirs(txt_folder_path, exist_ok=True)

    pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
    pdf_files = pdf_files[:max_files]  # 테스트용 파일 개수 제한

    if not pdf_files:
        print("⚠️ 변환할 PDF 파일이 없습니다!")
        return

    print(f"📝 총 {len(pdf_files)}개의 PDF 변환을 시작합니다...")

    # 🔥 멀티쓰레딩 적용 (속도 최적화)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        executor.map(lambda pdf: process_single_pdf(pdf, txt_folder_path), pdf_files)

    print(f"✅ 변환 완료! (저장 폴더: {txt_folder_path})")

# **경로 설정**
pdf_folder_path = './naver_reports_pdf'  # PDF 파일이 저장된 폴더
txt_folder_path = './txt_files'  # 변환된 TXT 파일을 저장할 폴더

# 실행 (5개 파일 테스트, num_workers 최적화)
process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=4, max_files=5)


📝 총 5개의 PDF 변환을 시작합니다...
✅ 변환 완료: ./txt_files\080407_대우증권_1.txt (소요 시간: 9.15초)
✅ 변환 완료: ./txt_files\080407_대우증권_2.txt (소요 시간: 9.99초)
✅ 변환 완료: ./txt_files\080414_대우증권_1.txt (소요 시간: 13.01초)
✅ 변환 완료: ./txt_files\080421_대우증권_1.txt (소요 시간: 29.90초)
✅ 변환 완료: ./txt_files\080401_대우증권_1.txt (소요 시간: 62.89초)
✅ 변환 완료! (저장 폴더: ./txt_files)


In [ ]:
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import os
import time
from concurrent.futures import ThreadPoolExecutor

# OCR을 위한 Tesseract 경로 설정 (Windows 사용 시 필요)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # 실제 설치 경로 입력

def pdf_to_txt(pdf_path, txt_path):
    """PDF를 텍스트로 변환 (OCR 포함)"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([page.extract_text() or "" for page in pdf.pages if page.extract_text()])

        # OCR 적용: 텍스트가 없으면 OCR 실행
        if not text.strip():
            print(f" {pdf_path}는 이미지 PDF일 가능성이 높음 → OCR 실행")
            text = ocr_pdf(pdf_path)

        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    except Exception as e:
        print(f" PDF 변환 오류: {pdf_path} - {e}")

def ocr_pdf(pdf_path):
    """OCR을 사용하여 이미지 PDF에서 텍스트 추출"""
    try:
        images = convert_from_path(pdf_path)
        text = "\n".join([pytesseract.image_to_string(img, lang='kor+eng', config='--psm 3 --oem 1') for img in images])
        return text
    except Exception as e:
        print(f" OCR 변환 오류: {pdf_path} - {e}")
        return ""

def process_single_pdf(pdf_path, txt_folder_path):
    """PDF 하나를 변환하는 함수"""
    txt_file_name = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(txt_folder_path, txt_file_name)

    start_time = time.time()
    pdf_to_txt(pdf_path, txt_path)
    end_time = time.time()

    print(f"변환 완료: {txt_path} (소요 시간: {end_time - start_time:.2f}초)")

def process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=8, max_files=8000):
    """PDF 폴더 내 모든 파일을 병렬 처리 (최대 8000개)"""
    os.makedirs(txt_folder_path, exist_ok=True)

    pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
    pdf_files = pdf_files[:max_files]  # 최대 8000개 파일 변환

    if not pdf_files:
        print(" 변환할 PDF 파일이 없습니다!")
        return

    print(f"총 {len(pdf_files)}개의 PDF 변환을 시작합니다...")

    # 멀티쓰레딩 적용 (속도 최적화)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        executor.map(lambda pdf: process_single_pdf(pdf, txt_folder_path), pdf_files)

    print(f" 변환 완료! (저장 폴더: {txt_folder_path})")

# **경로 설정**
pdf_folder_path = './naver_reports_pdf'  # PDF 파일이 저장된 폴더
txt_folder_path = './txt_files'  # 변환된 TXT 파일을 저장할 폴더

# 전체 실행 (최대 8000개 변환)
process_pdf_folder(pdf_folder_path, txt_folder_path, num_workers=8, max_files=8000)


총 8000개의 PDF 변환을 시작합니다...
변환 완료: ./txt_files\080407_대우증권_1.txt (소요 시간: 15.14초)
변환 완료: ./txt_files\080407_대우증권_2.txt (소요 시간: 19.89초)
변환 완료: ./txt_files\080414_대우증권_1.txt (소요 시간: 29.14초)
변환 완료: ./txt_files\080519_대우증권_1.txt (소요 시간: 26.24초)
변환 완료: ./txt_files\080520_대우증권_1.txt (소요 시간: 33.56초)
변환 완료: ./txt_files\080521_대우증권_1.txt (소요 시간: 27.64초)
변환 완료: ./txt_files\080428_대우증권_1.txt (소요 시간: 68.24초)
변환 완료: ./txt_files\080522_대우증권_1.txt (소요 시간: 32.46초)
변환 완료: ./txt_files\080513_대우증권_1.txt (소요 시간: 77.22초)
변환 완료: ./txt_files\080523_대우증권_1.txt (소요 시간: 25.25초)
변환 완료: ./txt_files\080526_대우증권_1.txt (소요 시간: 28.48초)
변환 완료: ./txt_files\080527_대우증권_1.txt (소요 시간: 22.69초)
변환 완료: ./txt_files\080421_대우증권_1.txt (소요 시간: 100.09초)
변환 완료: ./txt_files\080529_대우증권_1.txt (소요 시간: 26.75초)
변환 완료: ./txt_files\080528_대우증권_1.txt (소요 시간: 27.76초)
변환 완료: ./txt_files\080530_대우증권_1.txt (소요 시간: 28.51초)
변환 완료: ./txt_files\080602_대우증권_1.txt (소요 시간: 27.93초)
변환 완료: ./txt_files\080603_대우증권_1.txt (소요 시간: 31.28초)
변환 완료: ./txt_files\

Data-loss while decompressing corrupted data


변환 완료: ./txt_files\170329_하나증권_1.txt (소요 시간: 39.77초)
변환 완료: ./txt_files\170327_케이프투자증권_1.txt (소요 시간: 93.65초)
 PDF 변환 오류: ./naver_reports_pdf\170227_이베스트투자증권_1.pdf - bytes must be in range(0, 256)
변환 완료: ./txt_files\170227_이베스트투자증권_1.txt (소요 시간: 350.62초)


Data-loss while decompressing corrupted data


 PDF 변환 오류: ./naver_reports_pdf\170320_이베스트투자증권_1.pdf - can't concat float to bytes
변환 완료: ./txt_files\170320_이베스트투자증권_1.txt (소요 시간: 141.17초)
변환 완료: ./txt_files\170406_미래에셋증권_1.txt (소요 시간: 72.73초)
변환 완료: ./txt_files\170405_하나증권_1.txt (소요 시간: 80.49초)
변환 완료: ./txt_files\170407_이베스트투자증권_1.txt (소요 시간: 89.49초)
변환 완료: ./txt_files\170403_현대차증권_1.txt (소요 시간: 109.63초)
변환 완료: ./txt_files\170327_현대차증권_2.txt (소요 시간: 141.94초)
변환 완료: ./txt_files\170403_현대차증권_2.txt (소요 시간: 131.98초)
변환 완료: ./txt_files\170414_미래에셋증권_2.txt (소요 시간: 31.96초)
변환 완료: ./txt_files\170410_현대차증권_1.txt (소요 시간: 67.86초)
변환 완료: ./txt_files\170414_케이프투자증권_1.txt (소요 시간: 38.50초)
변환 완료: ./txt_files\170403_이베스트투자증권_1.txt (소요 시간: 196.63초)
변환 완료: ./txt_files\170414_미래에셋증권_3.txt (소요 시간: 48.89초)
변환 완료: ./txt_files\170410_키움증권_1.txt (소요 시간: 104.58초)
변환 완료: ./txt_files\170407_하나증권_1.txt (소요 시간: 207.14초)
변환 완료: ./txt_files\170417_IBK투자증권_1.txt (소요 시간: 47.28초)
변환 완료: ./txt_files\170414_미래에셋증권_1.txt (소요 시간: 167.35초)
변환 완료: ./txt_files\170417_현대차증

...

변환 완료: ./txt_files\181228_하나증권_1.txt (소요 시간: 73.52초)
변환 완료: ./txt_files\181212_하나증권_1.txt (소요 시간: 170.71초)
변환 완료: ./txt_files\181214_미래에셋증권_1.txt (소요 시간: 136.55초)
변환 완료: ./txt_files\181221_하나증권_1.txt (소요 시간: 115.66초)
변환 완료: ./txt_files\181212_하나증권_2.txt (소요 시간: 190.25초)
변환 완료: ./txt_files\181224_현대차증권_1.txt (소요 시간: 126.47초)
변환 완료: ./txt_files\190107_미래에셋증권_1.txt (소요 시간: 14.71초)
변환 완료: ./txt_files\190108_IBK투자증권_1.txt (소요 시간: 50.50초)
변환 완료: ./txt_files\190102_현대차증권_1.txt (소요 시간: 128.74초)
변환 완료: ./txt_files\190104_미래에셋증권_1.txt (소요 시간: 131.88초)
변환 완료: ./txt_files\190102_하나증권_1.txt (소요 시간: 201.19초)
변환 완료: ./txt_files\190104_하나증권_1.txt (소요 시간: 142.29초)
변환 완료: ./txt_files\190109_하나증권_2.txt (소요 시간: 86.26초)
변환 완료: ./txt_files\190114_현대차증권_1.txt (소요 시간: 74.04초)
변환 완료: ./txt_files\190109_하나증권_1.txt (소요 시간: 214.68초)
변환 완료: ./txt_files\190118_미래에셋증권_1.txt (소요 시간: 122.29초)
변환 완료: ./txt_files\190114_미래에셋증권_1.txt (소요 시간: 135.63초)
변환 완료: ./txt_files\190103_미래에셋증권_1.txt (소요 시간: 303.26초)
변환 완료: ./txt_fi

Data-loss while decompressing corrupted data


변환 완료: ./txt_files\190304_미래에셋증권_2.txt (소요 시간: 134.33초)
변환 완료: ./txt_files\190220_하나증권_1.txt (소요 시간: 312.21초)
변환 완료: ./txt_files\190308_미래에셋증권_1.txt (소요 시간: 131.29초)
변환 완료: ./txt_files\190325_미래에셋증권_1.txt (소요 시간: 25.28초)
변환 완료: ./txt_files\190315_미래에셋증권_1.txt (소요 시간: 135.22초)
변환 완료: ./txt_files\190318_하나증권_1.txt (소요 시간: 94.83초)
변환 완료: ./txt_files\190322_하나증권_1.txt (소요 시간: 89.04초)
변환 완료: ./txt_files\190409_미래에셋증권_1.txt (소요 시간: 16.38초)


Data-loss while decompressing corrupted data


변환 완료: ./txt_files\190322_미래에셋증권_1.txt (소요 시간: 154.86초)
변환 완료: ./txt_files\190313_하나증권_1.txt (소요 시간: 255.91초)
변환 완료: ./txt_files\190329_미래에셋증권_1.txt (소요 시간: 116.20초)
변환 완료: ./txt_files\190405_미래에셋증권_1.txt (소요 시간: 105.34초)
변환 완료: ./txt_files\190422_미래에셋증권_1.txt (소요 시간: 20.48초)
변환 완료: ./txt_files\190412_미래에셋증권_1.txt (소요 시간: 114.08초)
변환 완료: ./txt_files\190419_하나증권_1.txt (소요 시간: 105.91초)
변환 완료: ./txt_files\190410_미래에셋증권_1.txt (소요 시간: 153.25초)
변환 완료: ./txt_files\190419_미래에셋증권_1.txt (소요 시간: 139.28초)
변환 완료: ./txt_files\190426_미래에셋증권_1.txt (소요 시간: 113.28초)
변환 완료: ./txt_files\190509_미래에셋증권_1.txt (소요 시간: 43.78초)
변환 완료: ./txt_files\190426_하나증권_1.txt (소요 시간: 98.74초)
변환 완료: ./txt_files\190514_하나증권_1.txt (소요 시간: 98.12초)
변환 완료: ./txt_files\190515_하나증권_2.txt (소요 시간: 99.87초)
변환 완료: ./txt_files\190305_하나증권_1.txt (소요 시간: 581.95초)
변환 완료: ./txt_files\190508_하나증권_1.txt (소요 시간: 240.63초)
변환 완료: ./txt_files\190524_하나증권_1.txt (소요 시간: 117.49초)
변환 완료: ./txt_files\190430_미래에셋증권_1.txt (소요 시간: 271.36초)
변환 완료: ./txt_

✅ 민정 담당 기간(2023.03.01~2023.08.31) 데이터 저장 완료! (파일: ./filtered_minjeong.txt)


In [ ]:
import pdfplumber 
import pytesseract
from pdf2image import convert_from_path
import os
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# OCR을 위한 Tesseract 경로 설정 (Windows 사용 시 필요)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # 실제 설치 경로 입력

def pdf_to_txt(pdf_path, txt_path):
    """PDF를 텍스트로 변환 (OCR 포함)"""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([page.extract_text() or "" for page in pdf.pages if page.extract_text()])

        # OCR 적용: 텍스트가 없으면 OCR 실행
        if not text.strip():
            print(f" {pdf_path}는 이미지 PDF일 가능성이 높음 → OCR 실행")
            text = ocr_pdf(pdf_path)

        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    except Exception as e:
        print(f" PDF 변환 오류: {pdf_path} - {e}")

def ocr_pdf(pdf_path):
    """OCR을 사용하여 이미지 PDF에서 텍스트 추출"""
    try:
        images = convert_from_path(pdf_path)
        text = "\n".join([pytesseract.image_to_string(img, lang='kor+eng', config='--psm 3 --oem 1') for img in images])
        return text
    except Exception as e:
        print(f" OCR 변환 오류: {pdf_path} - {e}")
        return ""

def process_single_pdf(pdf_path, txt_folder_path):
    """PDF 하나를 변환하는 함수"""
    txt_file_name = os.path.splitext(os.path.basename(pdf_path))[0] + ".txt"
    txt_path = os.path.join(txt_folder_path, txt_file_name)

    start_time = time.time()
    pdf_to_txt(pdf_path, txt_path)
    end_time = time.time()

    print(f"변환 완료: {txt_path} (소요 시간: {end_time - start_time:.2f}초)")

def filter_pdfs_by_date(pdf_folder_path, start_date, end_date):
    """지정된 날짜 범위 내의 PDF 파일만 필터링"""
    pdf_files = [os.path.join(pdf_folder_path, f) for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]

    # 파일명이 'YYMMDD_...' 형식이라고 가정하고 필터링
    filtered_files = []
    for pdf in pdf_files:
        try:
            file_name = os.path.basename(pdf)
            file_date_str = file_name[:6]  # 앞 6자리 추출 (YYMMDD)
            
            # 'YYMMDD'를 '20YYMMDD'로 변환
            file_date = datetime.strptime("20" + file_date_str, "%Y%m%d")

            if start_date <= file_date <= end_date:
                filtered_files.append(pdf)
        except ValueError:
            continue  # 날짜 형식이 맞지 않으면 건너뜀

    return filtered_files

def process_pdf_folder(pdf_folder_path, txt_folder_path, start_date, end_date, num_workers=8):
    """PDF 폴더 내에서 특정 기간의 파일을 병렬 처리"""
    os.makedirs(txt_folder_path, exist_ok=True)

    # 지정된 날짜 범위의 파일만 필터링
    pdf_files = filter_pdfs_by_date(pdf_folder_path, start_date, end_date)

    if not pdf_files:
        print(f" {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')} 기간 내 변환할 PDF 파일이 없습니다!")
        return

    print(f"총 {len(pdf_files)}개의 PDF 변환을 시작합니다...")

    # 멀티쓰레딩 적용 (속도 최적화)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        executor.map(lambda pdf: process_single_pdf(pdf, txt_folder_path), pdf_files)

    print(f" 변환 완료! (저장 폴더: {txt_folder_path})")

# **경로 설정**
pdf_folder_path = './naver_reports_pdf'  # PDF 파일이 저장된 폴더
txt_folder_path = './txt_files'  # 변환된 TXT 파일을 저장할 폴더

# **변환할 날짜 범위 설정 (담당자가 직접 수정)**
start_date = datetime(2023, 3, 1)  # 시작 날짜 (예: 2023년 7월 24일)
end_date = datetime(2024, 2, 29)   # 종료 날짜 (예: 2023년 7월 25일)

# 전체 실행 (기간 내 파일만 변환)
process_pdf_folder(pdf_folder_path, txt_folder_path, start_date, end_date, num_workers=8)



총 18개의 PDF 변환을 시작합니다...
변환 완료: ./txt_files\230724_키움증권_1.txt (소요 시간: 4.58초)
변환 완료: ./txt_files\230724_교보증권_1.txt (소요 시간: 6.93초)
변환 완료: ./txt_files\230724_유진투자증권_1.txt (소요 시간: 7.01초)
변환 완료: ./txt_files\230724_신한투자증권_1.txt (소요 시간: 7.34초)
변환 완료: ./txt_files\230724_다올투자증권_1.txt (소요 시간: 7.52초)
변환 완료: ./txt_files\230724_키움증권_3.txt (소요 시간: 8.04초)
변환 완료: ./txt_files\230724_유안타증권_1.txt (소요 시간: 8.38초)
변환 완료: ./txt_files\230724_키움증권_2.txt (소요 시간: 8.50초)
변환 완료: ./txt_files\230725_다올투자증권_1.txt (소요 시간: 5.92초)
변환 완료: ./txt_files\230725_신한투자증권_1.txt (소요 시간: 6.25초)
변환 완료: ./txt_files\230724_한화투자증권_1.txt (소요 시간: 10.51초)
변환 완료: ./txt_files\230725_교보증권_1.txt (소요 시간: 8.39초)
변환 완료: ./txt_files\230725_유진투자증권_1.txt (소요 시간: 7.21초)
변환 완료: ./txt_files\230725_유안타증권_1.txt (소요 시간: 8.31초)
변환 완료: ./txt_files\230725_키움증권_1.txt (소요 시간: 8.51초)
